Wairakei-Tauhara Annual Seismic Report for Contact Energy
--

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib.transforms import offset_copy
from matplotlib import patheffects
import matplotlib.ticker as mticker

import numpy as np

import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import OSM

import warnings; warnings.simplefilter('ignore')

%matplotlib inline

**Functions for scale bar**

In [ ]:
def utm_from_lon(lon):
    """
    utm_from_lon - UTM zone for a longitude

    Not right for some polar regions (Norway, Svalbard, Antartica)

    :param float lon: longitude
    :return: UTM zone number
    :rtype: int
    """
    return np.floor( ( lon + 180 ) / 6) + 1

def scale_bar(ax, proj, length, location=(0.5, 0.05), linewidth=3,
              units='km', m_per_unit=1000):
    """

    http://stackoverflow.com/a/35705477/1072212
    ax is the axes to draw the scalebar on.
    proj is the projection the axes are in
    location is center of the scalebar in axis coordinates ie. 0.5 is the middle of the plot
    length is the length of the scalebar in km.
    linewidth is the thickness of the scalebar.
    units is the name of the unit
    m_per_unit is the number of meters in a unit
    """
    # find lat/lon center to find best UTM zone
    x0, x1, y0, y1 = ax.get_extent(proj.as_geodetic())
    # Projection in metres
    utm = ccrs.UTM(utm_from_lon((x0+x1)/2))
    # Get the extent of the plotted area in coordinates in metres
    x0, x1, y0, y1 = ax.get_extent(utm)
    # Turn the specified scalebar location into coordinates in metres
    sbcx, sbcy = x0 + (x1 - x0) * location[0], y0 + (y1 - y0) * location[1]
    # Generate the x coordinate for the ends of the scalebar
    bar_xs = [sbcx - length * m_per_unit/2, sbcx + length * m_per_unit/2]
    # buffer for scalebar
    buffer = [patheffects.withStroke(linewidth=5, foreground="w")]
    # Plot the scalebar with buffer
    ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
        linewidth=linewidth, path_effects=buffer)
    # buffer for text
    buffer = [patheffects.withStroke(linewidth=3, foreground="w")]
    # Plot the scalebar label
    t0 = ax.text(sbcx, sbcy, str(length) + ' ' + units, transform=utm,
        horizontalalignment='center', verticalalignment='bottom',
        path_effects=buffer, zorder=2)
    left = x0+(x1-x0)*0.05
    # Plot the N arrow
    t1 = ax.text(left, sbcy, u'\u25B2\nN', transform=utm,
        horizontalalignment='center', verticalalignment='bottom',
        path_effects=buffer, zorder=2)
    # Plot the scalebar without buffer, in case covered by text buffer
    ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
        linewidth=linewidth, zorder=3)
    
# or to use m instead of km
# scale_bar(ax, ccrs.Mercator(), 100000, m_per_unit=1, units='m')
# or to use miles instead of km
# scale_bar(ax, ccrs.Mercator(), 60, m_per_unit=1609.34, units='miles')

#need specific font to show triangle for north arrow
mpl.rc('font', family='DejaVu Sans')

In [ ]:
#notebook uses full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#box for search and map area
bbox = '175.85,-38.8,176.30,-38.45'
lonmin = 175.85
lonmax = 176.30
latmin = -38.8
latmax = -38.45

maxdepth = 20

**Edit this cell to set the current year for the report**

In [ ]:
startyear = '2016-01-01T00:00:00'
endyear = '2017-01-01T00:00:00'

#as well as annual data, show number of events since 1996
startall = '1996-01-01T00:00:00'

**Quake search, get all events since startall**

In [ ]:
url = 'http://quakesearch.geonet.org.nz/csv?bbox='+bbox+'&startdate='+startall+'&enddate='+endyear+'&maxdepth='+str(maxdepth)

In [ ]:
eqs = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime')

In [ ]:
eqs.head()

**Map for the reporting year**

In [ ]:
#select just events with an origintime (index) in the reporting year
eqsyr = eqs[(eqs.index>startyear)&(eqs.index<endyear)]

In [ ]:
#plot map
fig = plt.figure(dpi=200)

#imagery
imagery = OSM()
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 10)

#seismographs
names = ['lon', 'lat', 'code']
seis = pd.read_csv('seis.dat', sep='\s+', names=names)
plt.plot(seis['lon'], seis['lat'], color='blue', marker='^', markersize=4, linestyle='none', transform=ccrs.Geodetic())
#transform coordinate system so can plot text below symbol
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', y=-12)
#text labels
for index, row in seis.iterrows():
    plt.text(row['lon'], row['lat'], row['code'], verticalalignment='top', horizontalalignment='center', transform=text_transform, fontsize=4)
    
#power stations
names = ['lon', 'lat', 'let']
powst = pd.read_csv('powst.dat', sep='\s+', names=names)
plt.plot(powst['lon'], powst['lat'], color='black', marker='s', alpha=0.8, markersize=4, markeredgewidth=0, linestyle='none', transform=ccrs.Geodetic())
#transform coordinate system so can plot text below symbol
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', y=0)
#text labels
for index, row in powst.iterrows():
    plt.text(row['lon'], row['lat'], row['let'], verticalalignment='center', horizontalalignment='center', color='white', transform=text_transform, fontsize=4)
    
#huka falls and 7 km radius round huka falls
hukalon = 176.090
hukalat = -38.6488
plt.plot(hukalon, hukalat, color='gray', marker='s', alpha=0.8, markersize=5, markeredgewidth=0, linestyle='none', transform=ccrs.Geodetic())
plt.text(hukalon, hukalat, 'HF', verticalalignment='center', horizontalalignment='center', color='black', transform=text_transform, fontsize=4)
names = ['lon', 'lat']
hfrad = pd.read_csv('hukafalls7km.lonlat', sep='\s+', names=names)
plt.plot(hfrad['lon'], hfrad['lat'], color='black', alpha=0.5, linestyle='--', linewidth=0.5, transform=ccrs.Geodetic())

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
plt.scatter(eqsyr['longitude'], eqsyr[' latitude'], color='red', marker='o', s=eqsyr[' magnitude']**2, transform=ccrs.Geodetic())

scale_bar(ax, ccrs.Mercator(), 2)
plt.tight_layout()
plt.savefig('wairakei-tauhara_map.png', dpi=200)

**Numbers, etc for the reporting year**

In [ ]:
#total events in the "Wairakei area", corresponding to the map
eqsyr[' depth'].count()

In [ ]:
#events M>=3 so can give numbers
eqsyr[eqsyr[' magnitude']>=3]

In [ ]:
#minimum and maximum magnitude
min = eqsyr[' magnitude'].min()
max = eqsyr[' magnitude'].max()
print (min, max)

In [ ]:
#plot events M>=3 so can describe locations
fig = plt.figure(dpi=100)

#imagery
imagery = OSM()
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 10)

plt.scatter(eqsyr[eqsyr[' magnitude']>=3]['longitude'], eqsyr[eqsyr[' magnitude']>=3][' latitude'], color='red', marker='o', s=eqsyr[' magnitude']**2, transform=ccrs.Geodetic())

**Earthquakes per month since 1996**

In [ ]:
#by magnitude
eqsm2 = eqs[eqs[' magnitude']>=2]
eqsm3 = eqs[eqs[' magnitude']>=3]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3,ncols=1, sharex=True, figsize=(15,10))

mcount = eqsm3.resample('1M', label='left', closed='left').count()
ax1.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax1.bar(mcount.index, mcount.publicid, width = 10, color='blue', edgecolor='blue', align='edge', label='M>=3 earthquakes')
ax1.legend(loc='upper left')
ax1.set_ylim([0,3])
mcount = eqsm2.resample('1M', label='left', closed='left').count()
ax2.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax2.bar(mcount.index, mcount.publicid, width = 10, color='green', edgecolor='green', align='edge', label='M>=2 earthquakes')
ax2.legend(loc='upper left')
ax2.set_ylim([0,50])
mcount = eqs.resample('1M', label='left', closed='left').count()
ax3.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax3.bar(mcount.index, mcount.publicid, width = 10, color='red', edgecolor='red', align='edge', label='all earthquakes')
ax3.legend(loc='upper left')
ax3.set_ylim([0,220])
plt.autoscale(enable=True, axis='x', tight=True)#tight on x data limits
plt.tight_layout()
fig.savefig('wairakei-tauhara_monthly.png', dpi=200)

**Earthquakes per month within 7 km of Huka Falls**

In [ ]:
#vectorised numpy version of haversine formula to calculate distance between geographic points
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1 = np.radians(lon1)
    lon2 = np.radians(lon2)
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [ ]:
#add distance (in km) from huka falls to eqs dataframe
eqs['dist'] = haversine_np(hukalon, hukalat, eqs['longitude'], eqs[' latitude'])

In [ ]:
#by magnitude and distance from huka falls
eqshfm2 = eqs[(eqs[' magnitude']>=2)&(eqs['dist']<=7)]
eqshfm3 = eqs[(eqs[' magnitude']>=3)&(eqs['dist']<=7)]
eqshf = eqs[eqs['dist']<=7]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3,ncols=1, sharex=True, figsize=(15,10))

mcount = eqshfm3.resample('1M', label='left', closed='left').count()
ax1.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax1.bar(mcount.index, mcount.publicid, width = 10, color='blue', edgecolor='blue', align='edge', label='M>=3 earthquakes')
ax1.legend(loc='upper left')
ax1.set_ylim([0,3])
mcount = eqshfm2.resample('1M', label='left', closed='left').count()
ax2.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax2.bar(mcount.index, mcount.publicid, width = 10, color='green', edgecolor='green', align='edge', label='M>=2 earthquakes')
ax2.legend(loc='upper left')
ax2.set_ylim([0,50])
mcount = eqshf.resample('1M', label='left', closed='left').count()
ax3.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax3.bar(mcount.index, mcount.publicid, width = 10, color='red', edgecolor='red', align='edge', label='all earthquakes')
ax3.legend(loc='upper left')
ax3.set_ylim([0,220])
plt.autoscale(enable=True, axis='x', tight=True)#tight on x data limits
plt.tight_layout()
fig.savefig('wairakei-tauhara_monthly-hukafalls.png', dpi=200)